# Creación de conversaciones ICMP
Con este código se va a intentar crear todos los pasos de una conversación ICMP, mediante IA generativa. Se pretende pasar una descripción de esta conversación al modelo y que este sea capaz de crear todos los pasos/paquetes necesarios para replicar una conversación con esas mismas especificacioness

In [1]:
import os
import torch
from transformers import (
  AutoConfig,
  AutoTokenizer, 
  AutoModelForCausalLM, 
  BitsAndBytesConfig,
  GenerationConfig,
  pipeline
)

from langchain.embeddings.huggingface import HuggingFaceEmbeddings

from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [4]:
#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)
        

Your GPU supports bfloat16: accelerate training with bf16=True


In [5]:
model_name = 'mistralai/Mixtral-8x7B-Instruct-v0.1' #'mistralai/Mixtral-8x7B-Instruct-v0.1' #'mistralai/Codestral-22B-v0.1'

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, legacy=True)
    
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=compute_dtype,
    trust_remote_code=True,
    #device_map="auto",
    quantization_config=bnb_config,
)

generation_config = GenerationConfig.from_pretrained(model_name)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.1
generation_config.top_k = 10
generation_config.top_p = 0.1
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

model.generation_config.pad_token_ids = tokenizer.pad_token_id
    
# Crear LLM Chain
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=False,
    generation_config=generation_config,
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [7]:
prompt = PromptTemplate(
    input_variables=["system", "user"],
    output_parser=None,
    partial_variables={},
    template = """
    [INST]
    {system}
    
    {user}
    [/INST]
    """
)

system_message = "This is an ICMP Echo conversation summary:\n"
system_message += 'Source: IP="192.168.1.10" // Destination: IP="192.168.1.20"// Others: id=0x0045 seq= 81 ttl=56 type=Echo"\n'
system_message += "This is the python code to generate the request and reply packets of the conversation, respectively:\n"
system_message += 'pkt1=scapy.IP(src="192.168.1.10", dst="192.168.1.20", ttl=56)/scapy.ICMP(type=8, id=0x0045, seq=81)\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))'
system_message += 'pkt2=scapy.IP(src="192.168.1.20", dst="192.168.1.10", ttl=56)/scapy.ICMP(type=0, id=0x0045, seq=81)\n'

system_message = "This is an ICMP Timestamp conversation summary:\n"
system_message += 'Source: IP="222.240.4.204" // Destination: IP="192.168.1.20"// Others: id=0xFF62 seq= 4311 ttl=56 type=Timestamp"\n'
system_message += "This is the python code to generate the request and reply packets of the conversation, respectively:\n"
system_message += 'pkt1=scapy.IP(src="222.240.4.204", dst="200.40.184.21", ttl=56)/scapy.ICMP(type=13, id=0xFF62, seq=4311)\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))'
system_message += 'pkt2=scapy.IP(src="200.40.184.21", dst="222.240.4.204", ttl=56)/scapy.ICMP(type=14, id=0xFF62, seq=4311)\n'

In [8]:
with open("./data/Conversations/ICMP/Conv_summaries.txt","r",encoding="utf8") as f:
    Conv_summaries = f.read().splitlines()

print("Número de resúmenes: " +str(len(Conv_summaries)))

Número de resúmenes: 73


In [9]:
class code_response():
    "Stores name and place pairs"
    def __init__(self, name, place):
        self.prompt_summary = name
        self.completion = place

In [10]:
import pickle

try:
    responses = pickle.load(open("./data/Conversations/ICMP/pickle/ICMP_pairs_Aday.pkl", "rb" ))
except:
    responses = []
    
print("Number of previous responses: " + str(len(responses)))

Number of previous responses: 0


In [11]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)
rag_chain = prompt | mistral_llm

/home/ivangm/pytorch_env/pytorch_env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(


In [12]:
from scapy.all import *
from scapy.utils import RawPcapReader, wrpcap
import scapy.all as scapy

from scapy.layers.inet import IP, ICMP

from tqdm.auto import tqdm

progress_bar = tqdm(range(len(Conv_summaries)))

counter = 0

for i in range(len(Conv_summaries)):

    text_sum = ""
    
    summary = Conv_summaries[i]
    
    #print("\n...........................................")
    #print("GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter " +str(i+1) +")\n")
    
    query_content = "This is a conversation summary:\n"
    query_content += summary + "\n"
    text_sum += summary + "\n"
    query_content += "\nGenerate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code.Dont give back any comments or imports, just bare code and commands. Dont start the resposnses wiht ```python This is a flagrant error.\n"
    query_content += "Use the following structure:\n"
    query_content += "pkt1=...\n"
    query_content += "time.sleep(abs(random.gauss(0, 0.03)))\n"
    query_content += "pkt2=...\n"

    
    # Query es el mensaje que le envias al modelo
    #print(query_content)

    completion = ""
    completion = rag_chain.invoke({"system": system_message, "user": query_content})

    progress_bar.update(1)

    #print("\n...Generated!")

    # Filtramos ```python
    #pos1 = completion.find("```python\n")
    #pos2 = completion.find("\n```")
    #print(pos1, pos2)
    #if pos1 != -1 and pos2 != 1:
    #    completion = completion[pos1+len("```python\n"):pos2]

    # Parece que hay problemas de indentado en el código resultante ¿?
    completion_copy = completion
    completion = ""
    for line in completion_copy.splitlines():
        completion += line.strip()+"\n"
        
    # Guardamos tanto la petición como la respuesta en un archivo
    #pickle.dump(responses, open( "./data/Conversations/DNS/pickle/DNS_pairs_Aday.pkl", "wb" ) )
    try:
        #print("Ejecutando python...")
        exec(completion)
        #print(pkt1)
        #print(pkt2)
        counter += 1
    except:
        print("Error en la ejecución\n")
        print(completion)
        #print(":".join("{:02x}".format(ord(c)) for c in completion))
        continue

    # Guardamos tanto el resumen de los paquetes que solicitamos y lo que devuelve el modelo para estos paquetes
    try:
        responses.append(code_response(text_sum, completion))
    except:
        print("Error almacenando resultado del modelo\n")
    
    # Guardamos los paquetes generados en un pcap
    with open("./data/Conversations/ICMP/pcap/ICMP_Conv_generated_mixtral.pcap", "ba+") as f:
        wrpcap(f, pkt1, append=True)
    
    with open("./data/Conversations/ICMP/pcap/ICMP_Conv_generated_mixtral.pcap", "ba+") as f:
        wrpcap(f, pkt2, append=True)

print("Number of completitions done: " + str(counter))


  0%|          | 0/73 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting

Error en la ejecución


pkt1=scapy.IP(src="132.178.87.167", dst="211.239.188.225", ttl=DL)\
/scapy.ICMP(type=13, id=0x3E7A, seq=4981)
time.sleep(abs(random.gauss(0, 0.03)))
pkt2=scapy.IP(src="211.239.188.225", dst="132.178.87.167", ttl=DL)\
/scapy.ICMP(type=14, id=0x3E7A, seq=4981)



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Error en la ejecución


pkt1=scapy.IP(src="110.221.190.216", dst="195.11.171.134", ttl=DL)\
/scapy.ICMP(type=13, id=0x5E2A, seq=1569)
time.sleep(abs(random.gauss(0, 0.03)))
pkt2=scapy.IP(src="195.11.171.134", dst="110.221.190.216", ttl=DL)\
/scapy.ICMP(type=14, id=0x5E2A, seq=1569)



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Error en la ejecución


pkt1=scapy.IP(src="39.89.198.223", dst="171.109.55.123", ttl=DL)\
/scapy.ICMP(type=13, id=0x8B7D, seq=2416)
time.sleep(abs(random.gauss(0, 0.03)))
pkt2=scapy.IP(src="171.109.55.123", dst="39.89.198.223", ttl=DL)\
/scapy.ICMP(type=14, id=0x8B7D, seq=2416)



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Number of completitions done: 70


In [13]:
# Este es el resumen de lo que está en el struct de responses para la última petición
print(responses[-1].prompt_summary)
print(responses[-1].completion)




pkt1 = scapy.IP(src="222.240.4.204", dst="192.168.1.20", ttl=56)/scapy.ICMP(type=13, id=0xFF62, seq=4311)
time.sleep(abs(random.gauss(0, 0.03)))
pkt2 = scapy.IP(src="192.168.1.20", dst="222.240.4.204", ttl=56)/scapy.ICMP(type=14, id=0xFF62, seq=4311)

